In [ ]:
import pandas as pd
import numpy as np
import boto3
from splinter import Browser
from zipfile import ZipFile 
import os
import time
import requests
#sensitive variables in config.py file that is on .gitignore
from config import key_, secret_, s3_bucket, kaggle_cookie

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#look into using requests, and a login cookie vs splinter
url = 'http://www.kaggle.com'
browser.visit(url)
#you will have two minutes to manually login before the script will attempt to start downloading
time.sleep(120)

In [ ]:
def zip_file(number):
    num = str(number, s3, bucket=s3_bucket)
    if number < 10:
        url = f'https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_0{num}.zip'
        file_path = f'/Users/jeremiahherberg/Downloads/dfdc_train_part_0{num}.zip'
    else:
        url = f'https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_{num}.zip'
        file_path = f'/Users/jeremiahherberg/Downloads/dfdc_train_part_{num}.zip'
    browser.visit(url)
    time.sleep(420)
    try:
        with ZipFile(file_path, 'r') as zip:
            zip.extractall(path='../deepfake/videos/')
    except:
        time.sleep(120)
        try:
            with ZipFile(file_path, 'r') as zip:
                zip.extractall(path='../deepfake/videos/')
        except:
            time.sleep(180)
            with ZipFile(file_path, 'r') as zip:
                zip.extractall(path='../deepfake/videos/')
    meta_file = f'dfdc_train_part_{num}/metadata.json'
    with ZipFile(file_path, 'r') as zip:
        zip.extract(meta_file, path='../deepfake/data/')
    video_dir = f'videos/dfdc_train_part_{num}'
    video_list = os.listdir(video_dir)
    for video in video_list:
        s3.upload_file(f'{video_dir}/{video}', bucket, video)
        os.remove(f'{video_dir}/{video}')
    return file_path
        
        

In [ ]:
def combine_meta(num_start, num_end=0, aws_key=key_, aws_secret=secret_, bucket=s3_bucket):
    '''
    calls the zip file function and combines the meta files
    also uploads the new meta file to a bucket 
    '''
    if num_end == 0:
        num_end = num_start +1
    else:
        num_end +=1
    s3 = boto3.client('s3', aws_access_key_id=aws_key, aws_secret_access_key=aws_secret)
    meta_dictionary = {}
    for num in np.arange(num_start, num_end):
        path_of_zip_file = zip_file(num, s3)
        file_path = f'../deepfake/data/dfdc_train_part_{num}/metadata.json'
        with open (file_path) as meta:
            meta_data = json.load(meta)
        for video in meta_data:
            meta_dictionary[video] = meta_data[video]
        with open('meta.json', 'w') as d:
            json.dump(meta, d)
        os.remove(path_of_zip_file)
    #upload final meta_data file
    s3.upload_file('meta.json', bucket, 'meta.json')
    

In [ ]:
##
#don't believe this is needed anymore

# def get_meta_data(number):
#     num = str(number)
#     if number < 10:
#         url = f'https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_0{num}.zip'
#         file_path = f'/Users/jeremiahherberg/Downloads/dfdc_train_part_0{num}.zip'
#     else:
#         url = f'https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_{num}.zip'
#         file_path = f'/Users/jeremiahherberg/Downloads/dfdc_train_part_{num}.zip'
#     browser.visit(url)
#     time.sleep(420)
#     meta_file = f'dfdc_train_part_{num}/metadata.json'
#     try:
#         with ZipFile(file_path, 'r') as zip:
#             zip.extract(meta_file, path='../deepfake/data/')
#     except:
#         time.sleep(120)
#         try:
#             with ZipFile(file_path, 'r') as zip:
#                 zip.extract(meta_file, path='../deepfake/data/')
#         except:
#             time.sleep(180)
#             with ZipFile(file_path, 'r') as zip:
#                 zip.extract(meta_file, path='../deepfake/data/')
#     os.remove(file_path)

In [ ]:
combine_meta(0, 49)

In [ ]:
#need to unzip the "1" metadata file
#make sure the 0 file is uploaded
#same with the 1 and 2